# Calculating $\pi$ using QPE
* $\pi \approx 11.001001000011111101101010100010001000010110100011_2$.
* QPE can only measure phase information encoded under decimal point, i.e. $e^{2\pi i (2.1)}=e^{2\pi i(0.1)}$.
* Let's instead calculate $\pi/4$ and multiply the phase by 4 afterwards, which is equivalent to shifting the bits by 2.

<img src="token.png" width="450px" height="300px"></img><br/>

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

# token = 'MYTOKEN'
#QiskitRuntimeService.save_account(channel="ibm_quantum", token=token, overwrite=True)

from qiskit.providers.fake_provider import FakeGeneva
from qiskit_aer.noise import NoiseModel

fake_backend = FakeGeneva()  # 27-qubit device
noise_model = NoiseModel.from_backend(fake_backend)
backend = "ibmq_qasm_simulator"

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library import QFT
import numpy as np

In [ ]:
def create_qpe_circuit(theta, num_qubits):
    '''Creates a QPE circuit given theta and num_qubits.'''

    # Step 1: Create a circuit with two quantum registers and one classical register.
    first = QuantumRegister(size=num_qubits, name='first')  # the first register for phase estimation
    second = QuantumRegister(size=1, name='second')  # the second register for storing eigenvector |psi>
    classical = ClassicalRegister(size=num_qubits, name='readout') # classical register for readout
    qpe_circuit = QuantumCircuit(first, second, classical)

    # Step 2: Initialize the qubits.
    # All qubits are initialized in |0> by default, no extra code is needed to initialize the first register.
    qpe_circuit.x(second)  # Initialize the second register with state |psi>, which is |1> in this example.

    # Step 3: Create superposition in the first register.
    qpe_circuit.barrier()  # Add barriers to separate each step of the algorithm for better visualization.
    qpe_circuit.h(first)

    # Step 4: Apply a controlled-U^(2^j) black box.
    qpe_circuit.barrier()
    for j in range(num_qubits):
        qpe_circuit.cp(theta*2*np.pi*(2**j), j, num_qubits)  # Theta doesn't contain the 2 pi factor.

    # Step 5: Apply an inverse QFT to the first register.
    qpe_circuit.barrier()
    qpe_circuit.compose(QFT(num_qubits, inverse=True), inplace=True)

    # Step 6: Measure the first register.
    qpe_circuit.barrier()
    qpe_circuit.measure(first, classical)

    return qpe_circuit

In [ ]:
from qiskit.circuit import Parameter

num_qubits=10
theta = Parameter('theta')  # Create a parameter `theta` whose values can be assigned later.
qpe_circuit_parameterized = create_qpe_circuit(theta, num_qubits)

In [ ]:
phases = [np.pi/4]
individual_phases = [[ph] for ph in phases]  # Phases need to be expressed as a list of lists.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

In [ ]:
from qiskit_ibm_runtime import Session, Sampler, Options

options = Options(
    simulator={"noise_model": noise_model},
    resilience_level=1
)

with Session(service=service, backend=backend):
    job_id = Sampler(options=options).run(
        [qpe_circuit_parameterized]*len(individual_phases),
        parameter_values=individual_phases
    )
    results = job_id.result()

In [ ]:
print(job_id)

In [ ]:
job_id = 'cg7ncnn91ascc922orvg'

In [ ]:
service = QiskitRuntimeService(channel="ibm_quantum", instance='ibm-q-yonsei/externalq-meetup/tutorials')
results = service.job(job_id).result()

In [ ]:
pi_str = ""
probs = results.quasi_dists[0].binary_probabilities()
pi_str = pi_str + max(probs,key=probs.get)
print(pi_str)

In [ ]:
PI = "1100100100"

In [ ]:
PI == pi_str

In [ ]:
from qiskit.tools.visualization import plot_histogram

plot_histogram(results.quasi_dists[0].binary_probabilities(), bar_labels=False)

In [ ]:
probs = results.quasi_dists[0].binary_probabilities()
print(max(probs,key=probs.get))

Note: $\pi\approx 11.001001000011111101101010100010001000010110100011_2$

In [ ]:
# With M3
results = investigate_options(1)

In [ ]:
from qiskit.tools.visualization import plot_histogram

plot_histogram(results.quasi_dists[0].binary_probabilities(), bar_labels=False)

In [ ]:
probs = results.quasi_dists[0].binary_probabilities()
print(max(probs,key=probs.get))

Note: $\pi\approx 11.001001000011111101101010100010001000010110100011_2$

# Calculating 50 digits of $\pi$
This requires using 50 qubits, which is beyond the qubit count of our device (27).   
Let's instead chop the sequence of $\pi$ into 5 parts and "estimate" it using 10 qubits. We can run QPE 5 times.   
However, this may be slower than you'd expect.
<img src="ibm_sherbrooke.png" width="450px" height="300px"></img><br/>
5 minutes x 700 = 2.5 days!   
2.5 days x 5 = 12.5 days!!!   
Let's avoid being in the queue 5 times by actively using the Sampler primitive.

In [ ]:
phases = []
PI = "11001001000011111101101010100010001000010110100011"
for digits in np.arange(0,5):
    phases.append(int(str(PI[(digits*num_qubits):((digits+1)*num_qubits)]),2)/2**num_qubits)
individual_phases = [[ph] for ph in phases]  # Phases need to be expressed as a list of lists.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

In [ ]:
from qiskit_ibm_runtime import Session, Sampler, Options

options = Options(
    simulator={"noise_model": noise_model},
    resilience_level=1
)

with Session(service=service, backend=backend):
    job_id = Sampler(options=options).run(
        [qpe_circuit_parameterized]*len(individual_phases),
        parameter_values=individual_phases
    )
    results = job_id.result()

In [ ]:
pi_str = ""
for digits in np.arange(0,5):
    probs = results.quasi_dists[digits].binary_probabilities()
    pi_str = pi_str + max(probs,key=probs.get)
print(pi_str)

In [ ]:
PI == pi_str

# Calculating 50 digits of $\pi$ using ibm_sherbrooke
Since we have the access to the 127-qubit device **ibm_sherbrooke**, let's estimated the value of $\pi$ using 50 auxiliary qubits!

In [ ]:
# 1. Setting up a real device
from qiskit_ibm_provider import IBMProvider, IBMBackend

provider = IBMProvider(instance="ibm-q-yonsei/externalq-meetup/tutorials")
backend = provider.get_backend('ibm_sherbrooke')
service = QiskitRuntimeService(channel="ibm_quantum", instance='ibm-q-yonsei/externalq-meetup/tutorials')

In [ ]:
phases = [np.pi/4]
individual_phases = [[ph] for ph in phases]  # Phases need to be expressed as a list of lists.

In [ ]:
# With M3
options = Options(
    resilience_level=1
)

with Session(service=service, backend=backend):
    job_id = Sampler(options=options).run(
        [qpe_circuit_parameterized]*len(individual_phases),
        parameter_values=individual_phases
    )
    print(job_id)

In [ ]:
service = QiskitRuntimeService(channel="ibm_quantum", instance='ibm-q-yonsei/externalq-meetup/tutorials')
results = service.job(job_id).result()

In [ ]:
pi_str = ""
probs = results.quasi_dists[digits].binary_probabilities()
pi_str = pi_str + max(probs,key=probs.get)
print(pi_str)

In [ ]:
PI == pi_str